In [14]:
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
import copy

In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


Set up Data!

In [16]:
df = pd.read_csv('cardio_train.csv',sep=';')

In [17]:
features = df.drop(columns=['id','cardio']).values
labels = df['cardio'].values
scaler = StandardScaler()
features = scaler.fit_transform(features)


In [18]:
class CardioDataset(Dataset):
    def __init__(self, features, labels):
        self.x = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [19]:
dataset = CardioDataset(features, labels)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

Network Architecture

In [20]:
#network parameters
hyperParameters = {'crossover_rate':0.5,
                   'merge_rate': 0,
                   'mutation_rate':0.9,
                   'num_mutations':7,
                   'add_layer_rate':0.09,
                   'delete_layer_rate':0.07,
                   'randomize_weight_rate':0.16,
                   'randomize_bias_rate':0.14,
                   'randomize_beta_rate': 0.16,
                   'selection_type':'tournament',
                   'random_factor':0.1,
                   'num_best':0,
                   'population_size':6,
                   'num_generations':100}
nin = 11
nout = 2
beta = 0.95
spike_grad = surrogate.fast_sigmoid(slope=25)
num_steps = 100

In [21]:
class EONS:
    def __init__(self,params):
        self.template_network = None
        self.startLayer = None
        self.endLayer = None
        self.pop = None
        self.nin = None
        self.nout = None
        self.params = params
        pass

    def make_template_network(self,nin:int,nout:int):
        self.nin = nin
        self.nout = nout
        return None
    
    def initWeights(self,n):
        if isinstance(n,nn.Linear):
            nn.init.xavier_uniform_(n.weight)
            nn.init.zeros_(n.bias)
        

    def generatePopulation(self,):
        pop = []
        for i in range(self.params['population_size']):
            layerToGenerate = random.randint(2,10)
            neuronsPerLayer = [random.randint(50,1000) for i in range(layerToGenerate)]
            
            layers = [nn.Linear(self.nin,neuronsPerLayer[0]),snn.Leaky(beta=beta,spike_grad=spike_grad,init_hidden=True)]
            for j in range(layerToGenerate-1):
                layers.append(nn.Linear(neuronsPerLayer[j],neuronsPerLayer[j+1]))
                layers.append(snn.Leaky(beta=random.uniform(0.5,0.95),spike_grad=spike_grad,init_hidden=True))
            layers.append(nn.Linear(neuronsPerLayer[-1],self.nout))
            layers.append(snn.Leaky(beta=beta,spike_grad=spike_grad,init_hidden=True,output=True))
            net = nn.Sequential(*layers).to(device)
            net.apply(self.initWeights)
            net.metadata = {'layer_count':layerToGenerate,
                            'neurons': neuronsPerLayer}
            pop.append(net)
        self.pop = pop
        return pop
    
    def forward_pass(self,net,num_steps:int,data):
        mem_rec = []
        spk_rec = []
        utils.reset(net)

        for step in range(num_steps):
            spk_out,mem_out = net(data)
            spk_rec.append(spk_out)
            mem_rec.append(mem_out)
        return torch.stack(spk_rec),torch.stack(mem_rec)
    
    def fitness(self,net):
        net.eval()
        with torch.no_grad():
            total = 0
            acc = 0
            for batch_x,batch_y in dataloader:
                data = batch_x.to(device)
                targets = batch_y.to(device)
                output,_ = self.forward_pass(net,num_steps,data)

                acc += SF.accuracy_rate(output, targets) * output.size(1)
                total += output.size(1)
            bign = 1
            for i in range(0,len(net),2):
                layer = net[i]
                if hasattr(layer,"out_features"):
                    bign = max(bign,layer.out_features)
            w1,w2,w3 = 0.75,0.05,0.2
            wt = w1+w2+w3
            sizeNet = len(net) - 4
        return (acc/total * (w1/wt))+ (1/sizeNet * (w3/wt)) + (1/bign * (w2/wt))
    
    def tournamentSelection(self,fits): 
        best = []
        for i in range(len(fits)):
            tourney = random.sample(fits,3)
            tourney.sort(reverse=True)
            best.append(tourney[0])
        best.sort(reverse=True)
        return [fits.index(i) for i in best] # return indices of best fits
    
    def crossoverChild1(self,layerNet:nn.Sequential,weightNet:nn.Sequential):
        c1 = [i for i in layerNet]
        for i in range(len(c1)):
            if(isinstance(i,nn.Linear)):
                rows = min(weightNet[i].weight.size(0),c1[i].weight.size(0))
                cols = min(weightNet[i].weight.size(1),c1[i].weight.size(1))
                c1.weight.data[:rows,:cols] = weightNet.weight.data[:rows,:cols].clone()
                biaslen = min(weightNet[i].bias.size(0),c1[i].bias.size(0))
                c1.bias.data[:biaslen] = weightNet.bias.data[:biaslen].clone()
        return nn.Sequential(*c1)
    
    def crossoverChild2(self,layerNet:nn.Sequential,weightNet:nn.Sequential):
        c2 = [i for i in layerNet]
        for i in range(len(weightNet)):
            if(isinstance(i,nn.Linear)):
                rows = min(weightNet[i].weight.size(0),c2[i].weight.size(0))
                cols = min(weightNet[i].weight.size(1),c2[i].weight.size(1))
                c2.weight.data[:rows,:cols] = weightNet.weight.data[:rows,:cols].clone()
                biaslen = min(weightNet[i].bias.size(0),c2[i].bias.size(0))
                c2.bias.data[:biaslen] = weightNet.bias.data[:biaslen].clone()
        return nn.Sequential(*c2)
            
    def crossover(self,net1:nn.Sequential,net2:nn.Sequential):
        smallNet = net1
        bigNet = net2
        if(len(net1)>len(net2)):
            smallNet = net2
            bigNet = net1
        return self.crossoverChild1(smallNet,bigNet),self.crossoverChild2(bigNet,smallNet)
    
    def addLayer(nNet:nn.Sequential,cNet:nn.Sequential):
        pass
    
    def mutate(self,net:nn.Sequential):
        nNet = [net[0],net[1]]
        cNet = net
        lastLayer = [net[-2],net[-1]]
        with torch.no_grad():
            for i in range(2,len(cNet)-2):
                layer = cNet[i]
                if random.random() < self.params['add_layer_rate'] and isinstance(layer,nn.Linear):
                    pass
                if random.random() < self.params['delete_layer_rate'] and isinstance(layer,nn.Linear):
                    pass
                if random.random() < self.params['randomize_weight_rate'] and isinstance(layer,nn.Linear):
                    nn.init.xavier_normal_(layer.weight)
                if random.random() < self.params['randomize_bias_rate'] and isinstance(layer,nn.Linear):
                    layer.bias.uniform_(-1,1)
                if random.random() < self.params['randomize_beta_rate'] and isinstance(layer,snn.Leaky):
                    layer = snn.Leaky(beta=random.uniform(0.5,0.95),spike_grad=spike_grad,init_hidden=True)
                nNet.append(layer)
        for i in lastLayer:
            nNet.append(i)
        return nn.Sequential(*nNet)
    
    def reproduction(self,pop:list[nn.Sequential],fits:list):
        newPop = []
        for i in range(self.params['num_best']):
            newPop.append(pop[i])    
        for i in range(self.params['num_best'],len(fits),2):
            p1,p2 = pop[fits[i]],pop[fits[i+1]]
            c1,c2 = self.crossover(p1,p2)

            if random.random() < self.params['mutation_rate']:
                c1 = self.mutate(c1)
            if random.random() < self.params['mutation_rate']:
                c2 = self.mutate(c2)
            newPop.append(c1)
            newPop.append(c2)
        for i in range(len(newPop)):
            newPop[i] = newPop[i].to(device) 
        return newPop
    
    def getAvgFit(self,pop,results):
        fits = [pop[i] for i in results]
        return np.mean(fits)
    
        

In [22]:
eons = EONS(hyperParameters)
eons.make_template_network(nin,nout)

Setting up all populations

In [23]:
pop = eons.generatePopulation()

In [24]:
num_epoch = eons.params['num_generations']
for i in range(num_epoch):
    fits = [eons.fitness(net) for net in pop]
    results = eons.tournamentSelection(fits)
    avgFit = eons.getAvgFit(fits,results)
    print('Generation: '+str(i)+' Avg Fit:'+ str(round(avgFit,4)) +' Best Fit: '+ str(round(fits[results[0]],4)))
    pop = eons.reproduction(pop,results)
bestNet = pop[results[0]]

Generation: 0 Avg Fit:0.4248 Best Fit: 0.4554
Generation: 1 Avg Fit:0.4792 Best Fit: 0.4792
Generation: 2 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 3 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 4 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 5 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 6 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 7 Avg Fit:0.4748 Best Fit: 0.4748
Generation: 8 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 9 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 10 Avg Fit:0.4794 Best Fit: 0.4804
Generation: 11 Avg Fit:0.4909 Best Fit: 0.4961
Generation: 12 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 13 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 14 Avg Fit:0.4533 Best Fit: 0.4533
Generation: 15 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 16 Avg Fit:0.4757 Best Fit: 0.4757
Generation: 17 Avg Fit:0.5034 Best Fit: 0.5138
Generation: 18 Avg Fit:0.5319 Best Fit: 0.537
Generation: 19 Avg Fit:0.4753 Best Fit: 0.4753
Generation: 20 Avg Fit:0.4827 Best Fit: 0.4827
Generation: 21 Avg Fit:0

In [25]:
len(results)

6

In [26]:
with torch.no_grad():
    total = 0
    acc = 0
    for batch_x,batch_y in dataloader:
        data = batch_x.to(device)
        targets = batch_y.to(device)
        output,_ = eons.forward_pass(bestNet,num_steps,data)

        acc += SF.accuracy_rate(output, targets) * output.size(1)
        total += output.size(1)
    print(acc/total)

0.48824285714285715
